In [1]:
import numpy as np

# coordinate file
file = open("../datasets/2D/test_coordinates.txt","r")
# encoding file
file1 = open("../datasets/2D/test_encodings.txt","r")
# polygon bounding box file
# if not available assign ""
filename = ""


points = []
label = []
encoding = []

resolution = 32
points_per_scene = 200
input = 4
output = 1

while True:
    s = file.readline()
    if len(s)==0:
        break
    k = s[:-1].split(' ')[0:input]
    h = s[:-1].split(' ')[input:input+output]
    r = []
    for i in k:
        r.append(float(i))
    points.append(r)
    if (int(h[0])==0):
        label.append(0)
    else:
        label.append(1)

while True:
    s = file1.readline()
    if len(s)==0:
        break
    m = []
    for i in range (0,resolution):
        l = s[:-1].split(' ')[0:resolution]
        r = []
        for i in l:
            r.append(float(i))
        m.append(r)
        s = file1.readline()
    #for i in range(0,points_per_scene):
    encoding.append(m)

file.close()
file1.close()

p = np.array(points, dtype=float)
l = np.array(label, dtype=float)
scenes = np.array(encoding, dtype=float)

print(p.shape)
print(l.shape)
print(scenes.shape)

(2000, 4)
(2000,)
(10, 32, 32)


In [ ]:
import pygame

# Initialize Pygame
pygame.init()

# Define colors
WHITE = (255, 255, 255)
BLACK = (0,0,0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)

# Set up the display
WIDTH, HEIGHT = 512, 512
ROWS, COLS = scenes[0].shape
CELL_WIDTH, CELL_HEIGHT = WIDTH//ROWS, HEIGHT//COLS



DIS = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Visibility Map')

color_map = {
    0: WHITE,
    1: BLACK,
    2: BLUE,
    3: GREEN,
    4: RED
}

class Scene:
    def __init__(self, minX, maxX, minY, maxY, obstacles):
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.obstacles = obstacles


def read_obstacle_file(filename):
    obstacles = {}
    scenes = []
    with open(filename, 'r') as file:
        for line in file:
            if line.split()[0]=="a":
                type,scene_id,obstacle_id, vertex_id, y, x = line.split()
                scene_id = int(scene_id)
                obstacle_id = int(obstacle_id)
                x, y = float(x), float(y)
                x, y = x*WIDTH, y*HEIGHT
                if obstacle_id not in obstacles:
                    obstacles[obstacle_id] = []
                obstacles[obstacle_id].append((x, y))
            elif line.split()[0]=='b':
                type,min_x,max_x,min_y,max_y = line.split()
                min_x, max_x, min_y, max_y = float(min_x), float(max_x), float(min_y), float(max_y)
                scenes.append(Scene(min_x,max_x,min_y,max_y,obstacles))
                obstacles = {}
    return scenes

def draw_grid(data):
    for row in range(ROWS):
        for col in range(COLS):
            pygame.draw.rect(DIS, color_map[data[row][col]], (col * CELL_WIDTH, row * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT))
            
def get_cell_under_mouse(mouse_pos):
    x, y = mouse_pos
    col = x // CELL_WIDTH
    row = y // CELL_HEIGHT
    if row < ROWS and col < COLS:
        return (row, col)
    return None

def get_cor(x,y):
    return (y*WIDTH,x*HEIGHT)

if filename:
    scene_bound = read_obstacle_file(filename)
else:
    scene_bound = None

def gameLoop():
    game_over = False
    show_bounding_box = True
    show_encoding = True
    index = 0
    line_index = 0
    current_scene = scenes[index]
    data = current_scene
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                index = (index + 1)%scenes.shape[0]
                data = scenes[index]
                line_index = 0
                show_bounding_box = True
                show_encoding = True
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                index = (index - 1)%scenes.shape[0]
                data = scenes[index]
                line_index = 0
                show_bounding_box = True
                show_encoding = True
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_i:
                line_index = (line_index + 1)%points_per_scene
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_k:
                line_index = (line_index - 1)%points_per_scene
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_b:
                show_bounding_box = not show_bounding_box
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_e:
                show_encoding = not show_encoding


        # Fill the background
        DIS.fill(WHITE)

        # Draw the grid with the initial colors
        if show_encoding:
            draw_grid(data)
        if show_bounding_box and scene_bound!=None:
            for obstacle_id, vertices in scene_bound[index].obstacles.items():
                pygame.draw.polygon(DIS, color_map[2], vertices, width=2)

        if l[index*points_per_scene+line_index]==1:
            pygame.draw.line(DIS, (0,255,0), get_cor(p[index*points_per_scene+line_index][0],p[index*points_per_scene+line_index][1]), get_cor(p[index*points_per_scene+line_index][2],p[index*points_per_scene+line_index][3]), 2)
        elif l[index*points_per_scene+line_index]==0:
            pygame.draw.line(DIS, (255,0,0), get_cor(p[index*points_per_scene+line_index][0],p[index*points_per_scene+line_index][1]), get_cor(p[index*points_per_scene+line_index][2],p[index*points_per_scene+line_index][3]), 2)

        rendered_text = pygame.font.SysFont("Arial", 20).render(f"{index}", True, (255, 0, 0))
        DIS.blit(rendered_text, (WIDTH//2-20,10))
        rendered_text = pygame.font.SysFont("Arial", 20).render(f"{line_index}", True, (255, 0, 0))
        DIS.blit(rendered_text, (WIDTH//2+20,10))

        # Get the mouse position and check which cell is hovered
        mouse_pos = pygame.mouse.get_pos()
        hovered_cell = get_cell_under_mouse(mouse_pos)

        if hovered_cell:
            row, col = hovered_cell
            pygame.draw.rect(DIS, GRAY, (col * CELL_WIDTH, row * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT))

        pygame.display.update()

    pygame.quit()
    quit()

# Start the game loop
gameLoop()


pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 